In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import glob

In [2]:
#Cargando archivos del mapping
sntc_mapping = pd.read_excel(r"C:\Users\cmedinag\OneDrive - Nub78\Documentos\Tableau CM\Mapping Files\4-SNTC_Princing_List.xlsx",sheet_name="Output").fillna(0)
sntc_mapping = sntc_mapping[["Product SKU ","SNT"]]
sspt_mapping = pd.read_excel(r"C:\Users\cmedinag\OneDrive - Nub78\Documentos\Tableau CM\Mapping Files\3-SSPT_Princing_List.xlsx",sheet_name="Output Table").fillna(0)
sspte_mapping = pd.read_excel(r"C:\Users\cmedinag\OneDrive - Nub78\Documentos\Tableau CM\Mapping Files\3-SSPT_Princing_List.xlsx",sheet_name="Eligible SSPT").fillna(0)
ctl_mapping = pd.read_excel(r"C:\Users\cmedinag\OneDrive - Nub78\Documentos\Tableau CM\Mapping Files\1-Contract_Types_List.xlsx",sheet_name="SNTCSolutionSupport").fillna(0)
cs_mapping = pd.read_excel(r"C:\Users\cmedinag\OneDrive - Nub78\Documentos\Tableau CM\Mapping Files\8-Combined_Services.xlsx",sheet_name="Sheet1").fillna(0)
cs_mapping = cs_mapping[["Combined Services Service Level","Multiplier","Uplift",'Service Level']]

#Corrigiendo nombre de ECMU Y ECMUS en Contract Type List
for i in range(0,len(ctl_mapping)): 
    if (ctl_mapping.loc[i,'Contract Type'] =='\ufeffECMU'):
        ctl_mapping.loc[i,'Contract Type'] = 'ECMU'
    elif (ctl_mapping.loc[i,'Contract Type'] =='\ufeffECMUS'):
        ctl_mapping.loc[i,'Contract Type'] = 'ECMUS'
    else:
        ctl_mapping.loc[i,'Contract Type'] ==ctl_mapping.loc[i,'Contract Type']


# Functions

In [3]:
%%time

def LDoS_flag(data):
    list=[]
    for x in data:
        if x >= dt.today():
            list.append('Y')
        else:
            list.append('N')
    return list

# Funcion calcula el valor del SSPT_ESTIMATED
def SSPT_ESTIMATED(path, sntc_mapping, sspt_mapping, sspte_mapping, ctl_mapping, cs_mapping):
    '''
    La funcion recibe la ruta de un archivo excel y calcula el IB value
    entradas:
        path: Ruta del archivo.
    salidas:
        
    '''
    
 

    #Leyendo archivo de IB
    ib = pd.read_csv(path)


    #Haciendo merge a los archivos para traer las columnas del valor del ST y Contract Type a la tabla ib

    merge = ib.merge(sspt_mapping,right_on="Product SKU",left_on="Product ID",how="left")
    merge = merge.merge(sntc_mapping,right_on="Product SKU ",left_on="Product ID",how="left")
    merge = merge.merge(ctl_mapping,right_on="Contract Type",left_on="Contract Type",how="left")
    merge = merge.merge(sspte_mapping,right_on="Product SKU",left_on="Product ID",how="left")
    merge = merge.merge(cs_mapping,right_on="Combined Services Service Level",left_on="Contract Type",how="left")

    #Transformando las fechas para calcular la columna LDos flag
    merge['LDoS']=merge['LDoS'].apply(lambda x: str(x).replace('/','-'))    
    merge['LDoS']=merge['LDoS'].apply(lambda x: str(x).replace('nan', '2100-01-01 00:00:00'))
    fechas = [dt.strptime(str(i)[:-5],'%m-%d-%Y') if len(i)<16 else dt.fromisoformat(str(i)) for i in merge['LDoS']]
    merge['LDoS'] = fechas

    #Calculando columna LDos flag
    merge['LDoS Flag']=''
    merge['LDoS Flag']= LDoS_flag( merge['LDoS'])
    #merge[['LDoS','LDoS Flag']][merge['LDoS Flag']=='N']


    for i in range(0,len(merge)): 
        if (merge.loc[i,'LDoS Flag'] =='Y') & (merge.loc[i,'Covered Line Status']!='TERMINATED')& (merge.loc[i,'Eligible']=='YES') & (merge.loc[i,'Coverage']=='NOT COVERED'):
            merge.loc[i,'Select data to Display']='Uncovered Opportunity'
        elif  (merge.loc[i,'Eligible']=='YES') & (merge.loc[i,'Coverage']=='COVERED'):
            merge.loc[i,'Select data to Display']='Covered Opportunity'
        else:
            merge.loc[i,'Select data to Display'] = 'Uncovered IB to be Excluded'



    #Ejecutando los filtros a la tabla


    merge2=merge[(merge['Select data to Display'] == 'Covered Opportunity')]
    merge3=merge[(merge['Select data to Display'] == 'Uncovered Opportunity')]

    merge=pd.concat([merge2,merge3])

    merge=merge[merge['Coverage']=='COVERED']

    merge4=merge[(merge['SNTC SSPT Offer Flag']== 'CISCO SWSS')]
    merge5=merge[(merge['SNTC SSPT Offer Flag']=='Combined Services')]
    merge6=merge[(merge['SNTC SSPT Offer Flag']=='Partner Support')]
    merge7=merge[(merge['SNTC SSPT Offer Flag']=='Smartnet Total Care')]
    merge8=merge[(merge['SNTC SSPT Offer Flag']=='SP Base')]
    merge9=merge[(merge['SNTC SSPT Offer Flag']=='SS PLUS SWSS')] 
    merge11=merge[(merge['SNTC SSPT Offer Flag']=='TELEPRESENCE CUSTOMERS')]

    merge=pd.concat([merge4,merge5,merge6,merge7,merge8,merge9,merge11])


    #Success Track: All
    #Arquitecture: All
    #Product Family: All

    # Calculando - CS Multiplier SL
    merge.loc[i,'CS Multiplier SL'] = ''
    merge = merge.reset_index()

    for i in range(0,len(merge)):
        if (merge.loc[i,'Service Level'] == '24x7x2'):
            merge.loc[i,'CS Multiplier SL'] = merge.loc[i,'SSS2P']
        elif merge.loc[i,'Service Level'] == '24x7x2OS':
            merge.loc[i,'CS Multiplier SL'] = merge.loc[i,'SSC2P']
        elif merge.loc[i,'Service Level'] == '24x7x4':
            merge.loc[i,'CS Multiplier SL'] = merge.loc[i,'SSSNP']
        elif merge.loc[i,'Service Level'] == '24x7x4OS':
            merge.loc[i,'CS Multiplier SL'] = merge.loc[i,'SSC4P']   
        elif merge.loc[i,'Service Level'] == '8x5x4':
            merge.loc[i,'CS Multiplier SL'] = merge.loc[i,'SSSNE']     
        elif merge.loc[i,'Service Level'] == '8x5x4OS':
            merge.loc[i,'CS Multiplier SL'] = merge.loc[i,'SSC4S'] 
        elif merge.loc[i,'Service Level'] == 'NBD':
            merge.loc[i,'CS Multiplier SL'] = merge.loc[i,'SSSNT']
        elif merge.loc[i,'Service Level'] == '8x5xNDBOS':
            merge.loc[i,'CS Multiplier SL'] = merge.loc[i,'SSCS']
        elif merge.loc[i,'Service Level'] == 'DR 24x7x4OS':
            merge.loc[i,'CS Multiplier SL'] = merge.loc[i,'SSDR7']
        elif merge.loc[i,'Service Level'] == 'DR 8x5xNDBOS':
            merge.loc[i,'CS Multiplier SL'] = merge.loc[i,'SSDR5']
        elif merge.loc[i,'Service Level'] == 'SNTC NO RMA':
            merge.loc[i,'CS Multiplier SL'] = merge.loc[i,'SSSW']
        else:
            merge.loc[i,'CS Multiplier SL'] = 0

    # Calculando - CS SSPT (Multiplier)
    merge.loc[i,'CS SSPT (Multiplier)'] = 0


    for i in range(0,len(merge)): 
        if merge.loc[i,'Uplift'] =='':
            merge.loc[i,'CS SSPT (Multiplier)'] = 0
        else:
            merge.loc[i,'CS SSPT (Multiplier)'] = merge.loc[i,'CS Multiplier SL'] * merge.loc[i,'Item Quantity'] * merge.loc[i,'Multiplier']

    # Calculando - Uplift Recommended SL $

    merge['Uplift Recommended SL $']= ''
    lista = ['SNCO','PSUP','C2P','3C4P','C4P','C4S','CS','UCSD5','UCSD7','S2P','SNTE','SNTP','3SNT','5SNT','SNT','PSRT','PSUT','LSNT']
    lista2 = ['SPAR1','SPAR2','SPAR3','SPAR4','SPC2P','SBAR1','SPCS']
    lista3 = ['ECDN','ECDO']

    merge = merge.reset_index()

    for i in range(0,len(merge)):
        if (merge.loc[i,'Contract Type'] == 'C2P') & (merge.loc[i,'SSC2P'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSC2P']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'Contract Type'] == '3C4P') & (merge.loc[i,'SSC4P'] != 0):
                    merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSC4P']*merge.loc[i,'Item Quantity']    
        elif (merge.loc[i,'Contract Type'] == 'C4P') & (merge.loc[i,'SSC4P'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSC4P']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'Contract Type'] == 'C4S') & (merge.loc[i,'SSC4S'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSC4S']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'Contract Type'] == 'CS') & (merge.loc[i,'SSCS'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSCS']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'Contract Type'] == 'UCSD5') & (merge.loc[i,'SSDR5'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSDR5']*merge.loc[i,'Item Quantity']    
        elif (merge.loc[i,'Contract Type'] == 'UCSD7') & (merge.loc[i,'SSDR7'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSDR7']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'Contract Type'] == 'S2P') & (merge.loc[i,'SSS2P'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSS2P']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'Contract Type'] == 'SNTE') & (merge.loc[i,'SSSNE'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSSNE']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'Contract Type'] == '3SNTP') & (merge.loc[i,'SSSNP'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSSNP']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'Contract Type'] == 'SNTP') & (merge.loc[i,'SSSNP'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSSNP']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'Contract Type'] == '5SNTP') & (merge.loc[i,'SSSNP'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSSNP']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'Contract Type'] == 'SW') & (merge.loc[i,'SSSNT'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSSNT']*merge.loc[i,'Item Quantity']   
        elif (merge.loc[i,'Contract Type'] == 'SW') & (merge.loc[i,'SSSW'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSSW']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'Contract Type'] == 'ECMU') & (merge.loc[i,'ECMUS'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'ECMUS']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'SNTC SSPT Offer Flag'] == 'Combined Services'):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'CS SSPT (Multiplier)']   
        elif (merge.loc[i,'Contract Type'] == 'PSUP') & (merge.loc[i,'SSSNP'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSSNP']*merge.loc[i,'Item Quantity']    
        elif (merge.loc[i,'Contract Type'] == 'SNCO') & (merge.loc[i,'SSNCO'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSNCO']*merge.loc[i,'Item Quantity'] 
        elif (merge.loc[i,'Contract Type'] == 'SNC') & (merge.loc[i,'SSSNC'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSSNC']*merge.loc[i,'Item Quantity'] 
        elif (merge.loc[i,'Contract Type'] == 'SPAR2') & (merge.loc[i,'SNTC SSPT Offer Flag'] == 'SP Base')& (merge.loc[i,'SSSNE'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSSNE']*merge.loc[i,'Item Quantity']     
        elif (merge.loc[i,'Contract Type'] == 'SPAR3') & (merge.loc[i,'SNTC SSPT Offer Flag'] == 'SP Base')& (merge.loc[i,'SSSNP'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSSNP']*merge.loc[i,'Item Quantity'] 
        elif (merge.loc[i,'Contract Type'] == 'SPAR4') & (merge.loc[i,'SNTC SSPT Offer Flag'] == 'SP Base')& (merge.loc[i,'SSS2P'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSS2P']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'Contract Type'] == 'SPC2P') & (merge.loc[i,'SNTC SSPT Offer Flag'] == 'SP Base')& (merge.loc[i,'SSC2P'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSC2P']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'Contract Type'] == 'SPC4P') & (merge.loc[i,'SNTC SSPT Offer Flag'] == 'SP Base'):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSC4P']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'Contract Type'] == 'SPCS') & (merge.loc[i,'SNTC SSPT Offer Flag'] == 'SP Base')& (merge.loc[i,'SSCS'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSCS']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'Contract Type'] == 'ECDN') & (merge.loc[i,'SNTC SSPT Offer Flag'] == 'TELEPRESENCE CUSTOMERS'):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSSNT']*merge.loc[i,'Item Quantity']         
        elif (merge.loc[i,'Contract Type'] == 'ECDO') & (merge.loc[i,'SNTC SSPT Offer Flag'] == 'TELEPRESENCE CUSTOMERS'):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSCS']*merge.loc[i,'Item Quantity']          
        elif (merge.loc[i,'Contract Type'] in lista )& (merge.loc[i,'SSSNT'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSSNT']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'Contract Type'] in lista2) & (merge.loc[i,'SNTC SSPT Offer Flag'] == 'SP Base')& (merge.loc[i,'SSSNT'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSSNT']*merge.loc[i,'Item Quantity']    
        else:
            merge.loc[i,'Uplift Recommended SL $'] = 0

    # Calculando SNTC to SSPT Estimated $ Value Oppty
    #[SSPT Estimated $ Value Oppty]-sum([Default Service List Price])

    merge2 = merge

    total_SSPT_ESTIMATED1 = (merge2['Uplift Recommended SL $'].sum()) - (merge2['Annualized Extended Contract Line List USD Amount'].sum())



    #Cuando da negativo SNTC to SSPT Estimated $ Value Oppty

    if total_SSPT_ESTIMATED1 < 0:
        total_SSPT_ESTIMATED1 = (merge2['Uplift Recommended SL $'].sum()) - (merge2['Default Service List Price USD'].sum())
       
    return print('package # ',merge2.iloc[0,2].astype(int), ' done, SNTC to SSPT Oppty =', round(total_SSPT_ESTIMATED1/1000,1))

Wall time: 0 ns


# Testing

In [13]:
path = r'C:\Users\cmedinag\OneDrive - Nub78\Documentos\Tableau CM\OP\February\2022-02-04\*'


#El codigo extrae la url de cada imagen en el pc
listaarchivos = []
for file in glob.glob(path):
    print(file+'\Data\IB\IB.csv')
    listaarchivos.append(file+'\Data\IB\IB.csv')

C:\Users\cmedinag\OneDrive - Nub78\Documentos\Tableau CM\OP\February\2022-02-04\OP_APJC_SAV ID_203797997_Test13_2022_2_4\Data\IB\IB.csv
C:\Users\cmedinag\OneDrive - Nub78\Documentos\Tableau CM\OP\February\2022-02-04\OP_APJC_SAV ID_203892264_Test14_2022_2_4\Data\IB\IB.csv
C:\Users\cmedinag\OneDrive - Nub78\Documentos\Tableau CM\OP\February\2022-02-04\OP_APJC_SAV ID_283714970_Test11_2022_2_4\Data\IB\IB.csv
C:\Users\cmedinag\OneDrive - Nub78\Documentos\Tableau CM\OP\February\2022-02-04\OP_APJC_SAV ID_283715230_Test12_2022_2_4\Data\IB\IB.csv
C:\Users\cmedinag\OneDrive - Nub78\Documentos\Tableau CM\OP\February\2022-02-04\OP_APJC_SAV ID_283715236_Test10_2022_2_4\Data\IB\IB.csv


In [14]:
%%time
# Ahora se calculan todos lo IB value de los archivos mediante un ciclo
# Calculando todos los IB_Value en bache
#Las listas se generan afuera y en ellas se almacenan los datos calculados
lista_sspt=[]
lista_id=[]
for i in listaarchivos:
    SSPT_ESTIMATED(i, sntc_mapping, sspt_mapping, sspte_mapping, ctl_mapping, cs_mapping)


package #  203797997  done, SNTC to SSPT Oppty = 63560.7
package #  203892264  done, SNTC to SSPT Oppty = 21884.6
package #  283714970  done, SNTC to SSPT Oppty = 7048.3
package #  283715230  done, SNTC to SSPT Oppty = 18537.5
package #  283715236  done, SNTC to SSPT Oppty = 7359.3
Wall time: 10min 38s


In [64]:
#pasando los datos calculados a excel
SSPT_Oppty_calculated = pd.DataFrame(zip(lista_id,lista_sspt), columns=['ID', 'SNTC to SSPT Oppty'])
SSPT_Oppty_calculated.to_excel(r'SNTC to SSPT Oppty.xlsx')
SSPT_Oppty_calculated

,ID,SNTC to SSPT Oppty


# Prueba varios IDs


In [4]:
path = r'C:\Users\cmedinag\OneDrive - Nub78\Documentos\Initiatives\FY22Q2 - High Potential Acoounts\HPA DATA2\*'

lista_sav_IB= []

for file in glob.glob(path):
    lista_sav_IB.append(file)
    
SAV_IB =pd.DataFrame()

for i in lista_sav_IB:
    df =pd.read_csv(i,dtype={'SAV ID':str,
                            'Instance Shipped Fiscal Year':str,
                            'Item Quantity':float,
                            'LDoS FY':str,
                            'Best Site ID':int,
                            'Contract Type':str,
                            'Covered Line Status':str,
                            'Contract Number':str,
                            'Service Brand Code':str,
                            "Asset List Amount":float},decimal=",")
    df.insert(loc=2, column='Best Site ID', value=-9999)
    df.insert(loc=3, column='Best Site Customer Name', value='UNKNOWN')
    SAV_IB = pd.concat([df,SAV_IB], axis=0)
IB_full = SAV_IB
IB_full

,SAV ID,SAV Name,Best Site ID,Best Site Customer Name,Best Site Sales Level 2 Name,Coverage,Contract Number,Covered Line Status,Contract Type,Contract Line End Quarter,...,SWSS Flag,Default Service List Price USD,Item Quantity,Annual Extended Contract Line List USD Amount,Annual Contract Line Net USD Amount,Annualized Extended Contract Line List USD Amount,Annualized Contract Line Net USD Amount,Contract Line List Price USD,Contract Line Net Price USD,Asset List Amount
0,222173994,BELIZE TELECOM,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,512.01
1,222173994,BELIZE TELECOM,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,2589.44
2,222173994,BELIZE TELECOM,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,229.53
3,222173994,BELIZE TELECOM,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,74.0,NaN,NaN,NaN,NaN,NaN,NaN,13064.70
4,222173994,BELIZE TELECOM,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,423.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415803,283715230,SBU_PSR_MEX_CENTER_PSR105,-9999,UNKNOWN,LATIN AMERICA,COVERED,204218335,ACTIVE,PSUP,20251,...,N,165.0,1.0,479.85,86.276667,159.95,86.276667,479.85,258.83,2818.92
415804,283715230,SBU_PSR_MEX_CENTER_PSR105,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,3636.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,89863.95
415805,283715230,SBU_PSR_MEX_CENTER_PSR105,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,42889.88
415806,283715230,SBU_PSR_MEX_CENTER_PSR105,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,2882.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,60536.00


In [7]:
#IB_copy = IB_full.copy()
IB_copy = IB_full.copy()
lista_id = IB_copy.iloc[:,0].unique().tolist()


In [8]:
%%time

lista_oppty =[]
lista_sav_id = []
for j in lista_id:
    ib=IB_copy[IB_copy.iloc[:,0] == j]

    #Haciendo merge a los archivos para traer las columnas del valor del ST y Contract Type a la tabla ib

    merge = ib.merge(sspt_mapping,right_on="Product SKU",left_on="Product ID",how="left")
    merge = merge.merge(sntc_mapping,right_on="Product SKU ",left_on="Product ID",how="left")
    merge = merge.merge(ctl_mapping,right_on="Contract Type",left_on="Contract Type",how="left")
    merge = merge.merge(sspte_mapping,right_on="Product SKU",left_on="Product ID",how="left")
    merge = merge.merge(cs_mapping,right_on="Combined Services Service Level",left_on="Contract Type",how="left")

    #Transformando las fechas para calcular la columna LDos flag
    merge['LDoS']=merge['LDoS'].apply(lambda x: str(x).replace('/','-'))    
    merge['LDoS']=merge['LDoS'].apply(lambda x: str(x).replace('nan', '2100-01-01 00:00:00'))
    fechas = [dt.strptime(str(i)[:-5],'%m-%d-%Y') if len(i)<16 else dt.fromisoformat(str(i)) for i in merge['LDoS']]
    merge['LDoS'] = fechas

    #Calculando columna LDos flag
    merge['LDoS Flag']=''
    merge['LDoS Flag']= LDoS_flag( merge['LDoS'])
    #merge[['LDoS','LDoS Flag']][merge['LDoS Flag']=='N']
 

    for i in range(0,len(merge)): 
        if (merge.loc[i,'LDoS Flag'] =='Y') & (merge.loc[i,'Covered Line Status']!='TERMINATED')& (merge.loc[i,'Eligible']=='YES') & (merge.loc[i,'Coverage']=='NOT COVERED'):
            merge.loc[i,'Select data to Display']='Uncovered Opportunity'
        elif  (merge.loc[i,'Eligible']=='YES') & (merge.loc[i,'Coverage']=='COVERED'):
            merge.loc[i,'Select data to Display']='Covered Opportunity'
        else:
            merge.loc[i,'Select data to Display'] = 'Uncovered IB to be Excluded'


    #Ejecutando los filtros a la tabla


    merge2=merge[(merge['Select data to Display'] == 'Covered Opportunity')]
    merge3=merge[(merge['Select data to Display'] == 'Uncovered Opportunity')]

    merge=pd.concat([merge2,merge3])

    merge=merge[merge['Coverage']=='COVERED']

    merge4=merge[(merge['SNTC SSPT Offer Flag']== 'CISCO SWSS')]
    merge5=merge[(merge['SNTC SSPT Offer Flag']=='Combined Services')]
    merge6=merge[(merge['SNTC SSPT Offer Flag']=='Partner Support')]
    merge7=merge[(merge['SNTC SSPT Offer Flag']=='Smartnet Total Care')]
    merge8=merge[(merge['SNTC SSPT Offer Flag']=='SP Base')]
    merge9=merge[(merge['SNTC SSPT Offer Flag']=='SS PLUS SWSS')] 
    merge11=merge[(merge['SNTC SSPT Offer Flag']=='TELEPRESENCE CUSTOMERS')]

    merge=pd.concat([merge4,merge5,merge6,merge7,merge8,merge9,merge11])


    #Success Track: All
    #Arquitecture: All
    #Product Family: All

    # Calculando - CS Multiplier SL
    merge.loc[i,'CS Multiplier SL'] = ''
    merge = merge.reset_index()

    for i in range(0,len(merge)):
        if (merge.loc[i,'Service Level'] == '24x7x2'):
            merge.loc[i,'CS Multiplier SL'] = merge.loc[i,'SSS2P']
        elif merge.loc[i,'Service Level'] == '24x7x2OS':
            merge.loc[i,'CS Multiplier SL'] = merge.loc[i,'SSC2P']
        elif merge.loc[i,'Service Level'] == '24x7x4':
            merge.loc[i,'CS Multiplier SL'] = merge.loc[i,'SSSNP']
        elif merge.loc[i,'Service Level'] == '24x7x4OS':
            merge.loc[i,'CS Multiplier SL'] = merge.loc[i,'SSC4P']   
        elif merge.loc[i,'Service Level'] == '8x5x4':
            merge.loc[i,'CS Multiplier SL'] = merge.loc[i,'SSSNE']     
        elif merge.loc[i,'Service Level'] == '8x5x4OS':
            merge.loc[i,'CS Multiplier SL'] = merge.loc[i,'SSC4S'] 
        elif merge.loc[i,'Service Level'] == 'NBD':
            merge.loc[i,'CS Multiplier SL'] = merge.loc[i,'SSSNT']
        elif merge.loc[i,'Service Level'] == '8x5xNDBOS':
            merge.loc[i,'CS Multiplier SL'] = merge.loc[i,'SSCS']
        elif merge.loc[i,'Service Level'] == 'DR 24x7x4OS':
            merge.loc[i,'CS Multiplier SL'] = merge.loc[i,'SSDR7']
        elif merge.loc[i,'Service Level'] == 'DR 8x5xNDBOS':
            merge.loc[i,'CS Multiplier SL'] = merge.loc[i,'SSDR5']
        elif merge.loc[i,'Service Level'] == 'SNTC NO RMA':
            merge.loc[i,'CS Multiplier SL'] = merge.loc[i,'SSSW']
        else:
            merge.loc[i,'CS Multiplier SL'] = 0

    # Calculando - CS SSPT (Multiplier)
    merge.loc[i,'CS SSPT (Multiplier)'] = 0


    for i in range(0,len(merge)): 
        if merge.loc[i,'Uplift'] =='':
            merge.loc[i,'CS SSPT (Multiplier)'] = 0
        else:
            merge.loc[i,'CS SSPT (Multiplier)'] = merge.loc[i,'CS Multiplier SL'] * merge.loc[i,'Item Quantity'] * merge.loc[i,'Multiplier']

    # Calculando - Uplift Recommended SL $

    merge['Uplift Recommended SL $']= ''
    lista = ['SNCO','PSUP','C2P','3C4P','C4P','C4S','CS','UCSD5','UCSD7','S2P','SNTE','SNTP','3SNT','5SNT','SNT','PSRT','PSUT','LSNT']
    lista2 = ['SPAR1','SPAR2','SPAR3','SPAR4','SPC2P','SBAR1','SPCS']
    lista3 = ['ECDN','ECDO']

    merge = merge.reset_index()

    for i in range(0,len(merge)):
        if (merge.loc[i,'Contract Type'] == 'C2P') & (merge.loc[i,'SSC2P'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSC2P']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'Contract Type'] == '3C4P') & (merge.loc[i,'SSC4P'] != 0):
                    merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSC4P']*merge.loc[i,'Item Quantity']    
        elif (merge.loc[i,'Contract Type'] == 'C4P') & (merge.loc[i,'SSC4P'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSC4P']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'Contract Type'] == 'C4S') & (merge.loc[i,'SSC4S'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSC4S']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'Contract Type'] == 'CS') & (merge.loc[i,'SSCS'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSCS']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'Contract Type'] == 'UCSD5') & (merge.loc[i,'SSDR5'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSDR5']*merge.loc[i,'Item Quantity']    
        elif (merge.loc[i,'Contract Type'] == 'UCSD7') & (merge.loc[i,'SSDR7'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSDR7']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'Contract Type'] == 'S2P') & (merge.loc[i,'SSS2P'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSS2P']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'Contract Type'] == 'SNTE') & (merge.loc[i,'SSSNE'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSSNE']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'Contract Type'] == '3SNTP') & (merge.loc[i,'SSSNP'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSSNP']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'Contract Type'] == 'SNTP') & (merge.loc[i,'SSSNP'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSSNP']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'Contract Type'] == '5SNTP') & (merge.loc[i,'SSSNP'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSSNP']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'Contract Type'] == 'SW') & (merge.loc[i,'SSSNT'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSSNT']*merge.loc[i,'Item Quantity']   
        elif (merge.loc[i,'Contract Type'] == 'SW') & (merge.loc[i,'SSSW'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSSW']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'Contract Type'] == 'ECMU') & (merge.loc[i,'ECMUS'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'ECMUS']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'SNTC SSPT Offer Flag'] == 'Combined Services'):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'CS SSPT (Multiplier)']   
        elif (merge.loc[i,'Contract Type'] == 'PSUP') & (merge.loc[i,'SSSNP'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSSNP']*merge.loc[i,'Item Quantity']    
        elif (merge.loc[i,'Contract Type'] == 'SNCO') & (merge.loc[i,'SSNCO'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSNCO']*merge.loc[i,'Item Quantity'] 
        elif (merge.loc[i,'Contract Type'] == 'SNC') & (merge.loc[i,'SSSNC'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSSNC']*merge.loc[i,'Item Quantity'] 
        elif (merge.loc[i,'Contract Type'] == 'SPAR2') & (merge.loc[i,'SNTC SSPT Offer Flag'] == 'SP Base')& (merge.loc[i,'SSSNE'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSSNE']*merge.loc[i,'Item Quantity']     
        elif (merge.loc[i,'Contract Type'] == 'SPAR3') & (merge.loc[i,'SNTC SSPT Offer Flag'] == 'SP Base')& (merge.loc[i,'SSSNP'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSSNP']*merge.loc[i,'Item Quantity'] 
        elif (merge.loc[i,'Contract Type'] == 'SPAR4') & (merge.loc[i,'SNTC SSPT Offer Flag'] == 'SP Base')& (merge.loc[i,'SSS2P'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSS2P']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'Contract Type'] == 'SPC2P') & (merge.loc[i,'SNTC SSPT Offer Flag'] == 'SP Base')& (merge.loc[i,'SSC2P'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSC2P']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'Contract Type'] == 'SPC4P') & (merge.loc[i,'SNTC SSPT Offer Flag'] == 'SP Base'):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSC4P']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'Contract Type'] == 'SPCS') & (merge.loc[i,'SNTC SSPT Offer Flag'] == 'SP Base')& (merge.loc[i,'SSCS'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSCS']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'Contract Type'] == 'ECDN') & (merge.loc[i,'SNTC SSPT Offer Flag'] == 'TELEPRESENCE CUSTOMERS'):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSSNT']*merge.loc[i,'Item Quantity']         
        elif (merge.loc[i,'Contract Type'] == 'ECDO') & (merge.loc[i,'SNTC SSPT Offer Flag'] == 'TELEPRESENCE CUSTOMERS'):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSCS']*merge.loc[i,'Item Quantity']          
        elif (merge.loc[i,'Contract Type'] in lista )& (merge.loc[i,'SSSNT'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSSNT']*merge.loc[i,'Item Quantity']
        elif (merge.loc[i,'Contract Type'] in lista2) & (merge.loc[i,'SNTC SSPT Offer Flag'] == 'SP Base')& (merge.loc[i,'SSSNT'] != 0):
            merge.loc[i,'Uplift Recommended SL $'] = merge.loc[i,'SSSNT']*merge.loc[i,'Item Quantity']    
        else:
            merge.loc[i,'Uplift Recommended SL $'] = 0

    # Calculando SNTC to SSPT Estimated $ Value Oppty
    #[SSPT Estimated $ Value Oppty]-sum([Default Service List Price])

    merge2 = merge

    total_SSPT_ESTIMATED1 = (merge2['Uplift Recommended SL $'].sum()) - (merge2['Annualized Extended Contract Line List USD Amount'].sum())


    #Cuando da negativo SNTC to SSPT Estimated $ Value Oppty

    if total_SSPT_ESTIMATED1 < 0:
        total_SSPT_ESTIMATED1 = (merge2['Uplift Recommended SL $'].sum()) - (merge2['Default Service List Price USD'].sum())
        
    lista_oppty.append(round(total_SSPT_ESTIMATED1/1000,1))
    lista_sav_id.append(j)
    print('package # ',j, ' done, SNTC to SSPT Oppty =', round(total_SSPT_ESTIMATED1/1000,1))

package #  222173994  done, SNTC to SSPT Oppty = 500.5
package #  230366718  done, SNTC to SSPT Oppty = 433.5
package #  203840606  done, SNTC to SSPT Oppty = 777.1
package #  207593001  done, SNTC to SSPT Oppty = 570.0
package #  283468065  done, SNTC to SSPT Oppty = 580.0
package #  203772167  done, SNTC to SSPT Oppty = 542.0
package #  203772245  done, SNTC to SSPT Oppty = 421.6
package #  203711644  done, SNTC to SSPT Oppty = 458.7
package #  203849413  done, SNTC to SSPT Oppty = 526.4
package #  203701565  done, SNTC to SSPT Oppty = 556.5
package #  283536694  done, SNTC to SSPT Oppty = 439.3
package #  203743221  done, SNTC to SSPT Oppty = 496.1
package #  233445185  done, SNTC to SSPT Oppty = 1326.5
package #  203785620  done, SNTC to SSPT Oppty = 590.6
package #  203729711  done, SNTC to SSPT Oppty = 546.1
package #  203853367  done, SNTC to SSPT Oppty = 624.3
package #  203851345  done, SNTC to SSPT Oppty = 539.8
package #  203730139  done, SNTC to SSPT Oppty = 464.7
package #

In [9]:
df_sp_oppty = pd.DataFrame()
df_sp_oppty['ID'] = lista_sav_id
df_sp_oppty['ST_Oppty'] = lista_oppty

df_sp_oppty

,ID,ST_Oppty
0,222173994,500.5
1,230366718,433.5
2,203840606,777.1
3,207593001,570.0
4,283468065,580.0
...,...,...
1326,283714968,6745.8
1327,203797997,63560.7
1328,283715236,7357.5
1329,283714970,7049.2


In [10]:
df_sp_oppty.to_csv('SP_Oppty-5-2-2022.csv')